In [1]:
%cd ../

/app


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import mlflow
import pandas as pd
import matplotlib.pyplot as plt
import os
#os.environ["PATH"] = "/home/users/dmoreno2016/.TinyTeX/bin/x86_64-linux:" + os.environ["PATH"]

plt.rcParams.update({
    "text.usetex": True,
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "font.serif": ["Palatino"],
})


import warnings
warnings.filterwarnings('ignore')

In [3]:
mlflow.set_tracking_uri("file:./results/elasticc_1/ml-runs")

In [4]:
data_name =  'elasticc_1'
experiment_name = f"ft_classification/{data_name}/best_params"

experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment:
    experiment_id = experiment.experiment_id
    print(f"Experiment ID for '{experiment_name}': {experiment_id}")
else:
    print(f"Experiment '{experiment_name}' not found.")

runs = mlflow.search_runs(experiment_ids=[experiment_id])

order_columns = [
    'run_id',
    'status',
    'metrics.epoch',
    'metrics.f1/test',
    'metrics.loss/test',
    'params.imgs_params/fig_params/linewidth',
    'params.imgs_params/fig_params/markersize',
    'params.imgs_params/input_type',
    'params.imgs_params/use_err',
    'params.training/lr',
    'start_time',
    'end_time',
]

show_df = runs[order_columns]
show_df.loc[:, 'training_time'] = show_df['end_time'] - show_df['start_time']
cols = show_df.columns.tolist()
cols.insert(2, cols.pop(cols.index('training_time')))
show_df = show_df[cols].drop(['end_time', 'start_time'], axis=1)
show_df = show_df.sort_values(by=['metrics.f1/test'], ascending=False)

group_columns = [
    'params.imgs_params/fig_params/linewidth',
    'params.imgs_params/fig_params/markersize',
    'params.imgs_params/input_type',
    'params.imgs_params/use_err',
    'params.training/lr',
]

# Calculating the mean for metrics within each group
elasticc_1_stats = show_df.groupby(group_columns).agg({
    'metrics.f1/test': ['mean', 'std'],
    'metrics.loss/test': ['mean', 'std'],
    'training_time': ['mean', 'std']
}).reset_index()

# Adjusting the rename dictionary to handle tuple column names
rename_dict = {
    ('params.imgs_params/fig_params/linewidth', ''): 'Linewidth',
    ('params.imgs_params/fig_params/markersize', ''): 'Markersize',
    ('params.imgs_params/input_type', ''): 'Input Type',
    ('params.imgs_params/use_err', ''): 'Use Error',
    ('params.training/lr', ''): 'Learning Rate',
    ('metrics.f1/test', 'mean'): 'avg_f1/test',
    ('metrics.f1/test', 'std'): 'std_f1/test',
    ('metrics.loss/test', 'mean'): 'avg_loss/test',
    ('metrics.loss/test', 'std'): 'std_loss/test',
    ('training_time', 'mean'): 'avg_training_time',
    ('training_time', 'std'): 'std_training_time',
}

# Renaming columns using the adjusted dictionary
elasticc_1_stats.columns = [rename_dict.get(col, col) for col in elasticc_1_stats.columns]
elasticc_1_stats

Experiment ID for 'ft_classification/elasticc_1/best_params': 893539322704437943


,Linewidth,Markersize,Input Type,Use Error,Learning Rate,avg_f1/test,std_f1/test,avg_loss/test,std_loss/test,avg_training_time,std_training_time
0,0.5,2.0,overlay,True,5e-06,0.654629,0.003125,1.741529,0.170650,3 days 08:51:12.173800,0 days 12:05:52.932924425
1,2.0,5.0,6grid,True,5e-06,0.646481,0.002407,1.886240,0.191334,5 days 09:24:59.331500,1 days 23:03:00.229608197
